In [1]:
%matplotlib inline

In [2]:
from scipy.integrate import odeint,quad,dblquad,simps,quad_vec,nquad

In [3]:
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
import math
import threading
import scipy


from scipy import constants
from scipy import interpolate
from scipy import signal
from scipy import ndimage

# TESTING WITH 1D-ARRAY 

In [7]:
# x = np.array([1,2,3,4,5,6],dtype=complex)
# y = np.array([1,2,3,4,5,6], dtype=complex)
N=8
x = np.linspace(-N/2,N/2-1,N)
y = x.T


xx,yy = np.meshgrid(x,y)
func = np.exp(-1j*(xx**2 + yy**2))
xx_fft2 = np.fft.fft2(xx)

xx_fft2_all = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(xx)))


# print(xx)
# print('')
# print(yy)
# print('')
# print(xx_fft2)
# print('')
# print(xx_fft2_all)

In [22]:
x_shift = np.fft.fftshift(x)
xx_shift = np.fft.fftshift(xx)
yy_shift = np.fft.fftshift(yy)
func_fftshift = np.fft.fftshift(func)
np.allclose(func_fftshift, np.exp(-1j*(xx_shift**2 + yy_shift**2)))

True

In [38]:

f_x = np.zeros(Ns,dtype = complex)

for s in range(Ns):
    g_even = 0.0
    g_odd = 0.0     
    for t in range(Ns//2):
        w_even = np.exp(-2j*np.pi*(2*t*s/Ns))
        g_even += x_shift[2*t]*w_even
                
        w_odd = np.exp(-2j*np.pi*((2*t+1)*s/Ns))
        g_odd += x_shift[2*t+1]*w_odd
        
        f_x[s] = g_even + g_odd
#         f_x[s+Ns//2] = g_even - g_odd

In [39]:
np.allclose(f_x, np.fft.fft(np.fft.fftshift(x)))

True

In [40]:

Ns = len(x)
g = np.zeros((Ns,Ns),dtype = complex)
xx_fft = np.zeros((Ns,Ns),dtype = complex)



for x1 in range(Ns):
    for x2 in range(Ns):
        f_even = 0.0
        f_even1 = 0.0
        f_odd = 0.0
        f_odd1 = 0.0
        for s in range(Ns//2):
            g_even = 0.0
            g_odd = 0.0            
            g_even1 = 0.0
            g_odd1 = 0.0
            for t in range(Ns//2):
                w_even = np.exp(-2j*np.pi*(2*t*x2/Ns))
                g_even += xx_shift[s,2*t]*w_even
                g_even1 += xx_shift[s+Ns//2,2*t]*w_even
                
                w_odd = np.exp(-2j*np.pi*((2*t+1)*x2/Ns))
                g_odd += xx_shift[s,2*t+1]*w_odd
                g_odd1 += xx_shift[s+Ns//2,2*t+1]*w_odd
            
            g[s,x2] = g_even + g_odd
            
            g[s+Ns//2,x2] = g_even1+g_odd1
            
            f_even += g[s,x2]*np.exp(-2j*np.pi*(s*x1/Ns))
            
            f_even1 += g[s+Ns//2,x2]*np.exp(-2j*np.pi*((s+Ns//2)*x1/Ns))
            
        xx_fft[x1,x2] = f_even+f_even1
        
    print(x1)

0
1
2
3
4
5
6
7


In [37]:
np.allclose(xx_fft,xx_fft2)

True

In [12]:

Ns = len(x)
g = np.zeros((Ns,Ns),dtype = complex)
xx_fft = np.zeros((Ns,Ns),dtype = complex)



for x1 in range(Ns):
    for x2 in range(Ns):
        f_even = 0.0
        f_even1 = 0.0
        f_odd = 0.0
        f_odd1 = 0.0
        for s in range(Ns//8):
            g_even = 0.0
            g_odd = 0.0            
            g_even1 = 0.0
            g_odd1 = 0.0
            for t in range(Ns//8):
                
                w_1 = np.exp(-2j*np.pi*(8*t*x2/Ns))
                w_2 = np.exp(-2j*np.pi*((8*t+1)*x2/Ns))
                w_3 = np.exp(-2j*np.pi*((8*t+2)*x2/Ns))
                w_4 = np.exp(-2j*np.pi*((8*t+3)*x2/Ns))
                w_5 = np.exp(-2j*np.pi*((8*t+4)*x2/Ns))
                w_6 = np.exp(-2j*np.pi*((8*t+5)*x2/Ns))
                w_7 = np.exp(-2j*np.pi*((8*t+6)*x2/Ns))
                w_8 = np.exp(-2j*np.pi*((8*t+7)*x2/Ns))
                
                g_even += (xx_shift[:,8*t]*w_1 + xx_shift[:,8*t+1]*w_2 + 
                           xx_shift[:,8*t+2]*w_3 + xx_shift[:,8*t+3]*w_4 +
                           xx_shift[:,8*t+4]*w_5 + xx_shift[:,8*t+5]*w_6 + 
                           xx_shift[:,8*t+6]*w_7 + xx_shift[:,8*t+7]*w_8)
            
            g[:,x2] = g_even
            
            
            f_even += (g[8*s,x2]*np.exp(-2j*np.pi*(8*s*x1/Ns))
                       +g[8*s+1,x2]*np.exp(-2j*np.pi*((8*s+1)*x1/Ns))
                       +g[8*s+2,x2]*np.exp(-2j*np.pi*((8*s+2)*x1/Ns))
                       +g[8*s+3,x2]*np.exp(-2j*np.pi*((8*s+3)*x1/Ns))
                       +g[8*s+4,x2]*np.exp(-2j*np.pi*((8*s+4)*x1/Ns))
                       +g[8*s+5,x2]*np.exp(-2j*np.pi*((8*s+5)*x1/Ns))
                       +g[8*s+6,x2]*np.exp(-2j*np.pi*((8*s+6)*x1/Ns))
                       +g[8*s+7,x2]*np.exp(-2j*np.pi*((8*s+7)*x1/Ns)))
            
        xx_fft[x1,x2] = f_even
    print(x1)

0
1
2
3
4
5
6
7


In [13]:
np.allclose(xx_fft,xx_fft2)

False

In [14]:
np.allclose(np.fft.ifftshift(xx_fft),xx_fft2_all)

True

In [59]:
Ns = len(arr)
fft_arr = np.zeros(Ns,dtype = complex)
arr_shift = np.fft.fftshift(arr)

for k in range(Ns):
    fft_sum_even = 0.0
    fft_sum_odd = 0.0
    for n in range(Ns//2):
        w_even = np.exp(-1j*2.*np.pi*float(k*2*n)/Ns)
        arr_x_even = arr_shift[2*n]
        fft_sum_even += arr_x_even*w_even
        
        w_odd = np.exp(-1j*2*np.pi*float(k*(2*n+1))/Ns)
        arr_x_odd = arr_shift[2*n+1]
        fft_sum_odd += arr_x_odd*w_odd
        
    fft_arr[k] = fft_sum_even + fft_sum_odd
    print(k)
    
    
print(fft_arr)
fft_arr_ishift = np.fft.ifftshift(fft_arr)
print(fft_arr_ishift)
# print(arr_shift)

0
1
2
3
4
5
6
7
[36.+0.00000000e+00j  4.-9.65685425e+00j -4.+4.00000000e+00j
  4.-1.65685425e+00j -4.-3.91886976e-15j  4.+1.65685425e+00j
 -4.-4.00000000e+00j  4.+9.65685425e+00j]
[-4.-3.91886976e-15j  4.+1.65685425e+00j -4.-4.00000000e+00j
  4.+9.65685425e+00j 36.+0.00000000e+00j  4.-9.65685425e+00j
 -4.+4.00000000e+00j  4.-1.65685425e+00j]
